In [70]:
import numpy as np
import pandas as pd
import networkx as nx



import os
import sys
import inspect
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0, parentdir)

sys.path.insert(0, '..')

import Utils as Utils

import scanpy as sc

import mygene

In [72]:
data_path = "scRNA-seq-A Molecular Switch from Tumor Suppressor to Oncogene in ER-Positive Breast Cancer - Role of Androgen Receptor, JAK-STAT, and Lineage Plasticity/";
file_name = "";

adata = sc.read_10x_mtx(data_path+file_name);
adata


AnnData object with n_obs × n_vars = 34425 × 19912
    var: 'gene_ids', 'feature_types'

In [74]:
adata.var["mito"] = adata.var_names.str.startswith("MT-");
sc.pp.filter_cells(adata, min_genes=4500);
sc.pp.filter_genes(adata, min_cells=1000);
adata

AnnData object with n_obs × n_vars = 2608 × 6411
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells'

In [76]:
# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)


In [78]:
sc.pp.highly_variable_genes(adata, n_top_genes=300, inplace=True);
adata


AnnData object with n_obs × n_vars = 2608 × 6411
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [80]:
adata.var.highly_variable
adata.X
print(adata)
print(adata.X)
display(adata.obs)
display(adata.var)

AnnData object with n_obs × n_vars = 2608 × 6411
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'mito', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
<Compressed Sparse Column sparse matrix of dtype 'float32'
	with 11002756 stored elements and shape (2608, 6411)>
  Coords	Values
  (2, 0)	0.7780241966247559
  (9, 0)	0.9978222250938416
  (11, 0)	0.9452248811721802
  (18, 0)	0.8068850636482239
  (21, 0)	0.8763114809989929
  (23, 0)	0.8622035384178162
  (24, 0)	0.8479362726211548
  (25, 0)	0.7422558069229126
  (26, 0)	0.9859579205513
  (49, 0)	1.1659784317016602
  (50, 0)	0.7509225010871887
  (57, 0)	0.8965767025947571
  (59, 0)	0.7342236042022705
  (61, 0)	0.7148780822753906
  (64, 0)	0.7521486878395081
  (65, 0)	1.0629810094833374
  (68, 0)	0.727285623550415
  (72, 0)	0.7750038504600525
  (77, 0)	0.6022589206695557
  (81, 0)	0.944959282875061
  (87, 0)	0.9217529296875
  (88, 0)	0.9944010376930237
  (97, 0)	0.8286004662513733
  

,n_genes
AAACGCTTCGATACAC-1,4563
AACCAACAGCACCTGC-1,5177
AACGGGACAGCCCAGT-1,5149
AATGGCTCACTTCAGA-1,6643
AATTTCCCACGCGGTT-1,4688
...,...
TTTGACTCAGCCTATA-9,5498
TTTGACTGTATACAGA-9,5349
TTTGGAGAGTGATAGT-9,5682
TTTGGAGAGTGCAGCA-9,4994


,gene_ids,feature_types,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
SAMD11,ENSG00000187634,Gene Expression,False,1053,False,0.440527,0.271237,-0.243047
NOC2L,ENSG00000188976,Gene Expression,False,1007,False,0.392768,0.204924,-0.564913
HES4,ENSG00000188290,Gene Expression,False,1372,False,0.758175,0.989821,1.506566
AGRN,ENSG00000188157,Gene Expression,False,1624,False,0.735356,0.377792,-0.356361
C1orf159,ENSG00000131591,Gene Expression,False,1729,False,0.801177,0.335256,-0.485832
...,...,...,...,...,...,...,...,...
MT-ND4L,ENSG00000212907,Gene Expression,True,2101,False,2.229797,2.448728,1.093428
MT-ND4,ENSG00000198886,Gene Expression,True,2513,False,3.802875,3.992980,-0.214641
MT-ND5,ENSG00000198786,Gene Expression,True,2326,False,2.902728,3.092417,0.327394
MT-ND6,ENSG00000198695,Gene Expression,True,1795,False,1.597490,1.893883,1.213555


In [82]:
file_name = "filtereddata";

adata.write_csvs(data_path+file_name, skip_data=False, sep=',')

In [83]:
obs = pd.read_csv(data_path+file_name+"/obs.csv",index_col=0);
display(obs);
var = pd.read_csv(data_path+file_name+"/var.csv");
display(var);

dataset = pd.read_csv(data_path+file_name+"/X.csv",names=var['gene_ids']);
dataset.index = obs.index;
display(dataset);

highly_variable_genes = var[var['highly_variable']]['gene_ids'].values;
dataset = dataset[highly_variable_genes];
display(dataset);

,n_genes
AAACGCTTCGATACAC-1,4563
AACCAACAGCACCTGC-1,5177
AACGGGACAGCCCAGT-1,5149
AATGGCTCACTTCAGA-1,6643
AATTTCCCACGCGGTT-1,4688
...,...
TTTGACTCAGCCTATA-9,5498
TTTGACTGTATACAGA-9,5349
TTTGGAGAGTGATAGT-9,5682
TTTGGAGAGTGCAGCA-9,4994


,Unnamed: 0,gene_ids,feature_types,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
0,SAMD11,ENSG00000187634,Gene Expression,False,1053,False,0.440527,0.271237,-0.243047
1,NOC2L,ENSG00000188976,Gene Expression,False,1007,False,0.392768,0.204924,-0.564913
2,HES4,ENSG00000188290,Gene Expression,False,1372,False,0.758175,0.989821,1.506566
3,AGRN,ENSG00000188157,Gene Expression,False,1624,False,0.735356,0.377792,-0.356361
4,C1orf159,ENSG00000131591,Gene Expression,False,1729,False,0.801177,0.335256,-0.485832
...,...,...,...,...,...,...,...,...,...
6406,MT-ND4L,ENSG00000212907,Gene Expression,True,2101,False,2.229797,2.448728,1.093428
6407,MT-ND4,ENSG00000198886,Gene Expression,True,2513,False,3.802875,3.992980,-0.214641
6408,MT-ND5,ENSG00000198786,Gene Expression,True,2326,False,2.902728,3.092417,0.327394
6409,MT-ND6,ENSG00000198695,Gene Expression,True,1795,False,1.597490,1.893883,1.213555


,ENSG00000187634,ENSG00000188976,ENSG00000188290,ENSG00000188157,ENSG00000131591,ENSG00000078808,ENSG00000160087,ENSG00000131584,ENSG00000127054,ENSG00000221978,...,ENSG00000198712,ENSG00000228253,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727
AAACGCTTCGATACAC-1,0.000000,0.000000,0.913277,0.000000,0.000000,0.000000,0.913277,0.913277,0.000000,1.941602,...,1.941602,0.000000,1.941602,2.939676,2.939676,0.913277,0.913277,0.913277,0.000000,2.298061
AACCAACAGCACCTGC-1,0.000000,0.000000,1.526375,0.000000,1.526375,1.224063,0.788680,0.000000,0.000000,1.224063,...,5.495112,2.944825,5.407578,5.915530,5.116001,2.653621,4.428836,3.578344,2.361223,4.881690
AACGGGACAGCCCAGT-1,0.778024,0.000000,1.511053,0.778024,0.000000,0.000000,1.511053,1.511053,0.000000,1.210253,...,2.716415,0.000000,2.716415,3.200440,2.987431,1.210253,1.511053,0.778024,0.000000,2.223560
AATGGCTCACTTCAGA-1,0.000000,0.000000,1.246623,0.975459,0.000000,0.000000,0.975459,0.000000,0.602233,1.784593,...,4.992037,2.390075,5.072852,5.673301,4.715904,2.863563,4.242391,3.249823,2.654585,4.631049
AATTTCCCACGCGGTT-1,0.000000,0.000000,0.862805,0.000000,0.000000,0.000000,0.000000,0.862805,0.000000,0.000000,...,4.515318,2.858627,4.811469,5.261507,4.641838,2.776816,3.918258,3.070235,0.000000,4.137418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGACTCAGCCTATA-9,0.000000,0.599484,0.000000,0.971672,0.971672,0.000000,0.000000,0.599484,0.000000,0.599484,...,2.220485,0.000000,1.455055,1.909284,1.242291,0.599484,0.599484,0.000000,0.599484,0.599484
TTTGACTGTATACAGA-9,0.000000,0.000000,0.000000,0.000000,2.166122,0.000000,0.000000,1.489090,0.000000,0.619643,...,4.153451,1.947031,3.927490,4.348800,3.253844,1.273907,3.072617,2.424730,1.816385,3.462528
TTTGGAGAGTGATAGT-9,0.592462,0.000000,0.000000,0.000000,0.961979,0.592462,0.000000,1.443086,0.000000,0.961979,...,5.671328,2.794856,5.444165,5.944826,4.765712,3.015655,4.576810,3.723806,1.766546,4.799559
TTTGGAGAGTGCAGCA-9,0.000000,0.784452,0.784452,0.784452,0.784452,0.000000,0.784452,0.000000,0.784452,1.218589,...,2.646407,0.000000,2.558162,2.727492,1.939609,0.784452,0.784452,0.000000,0.000000,0.784452


,ENSG00000142676,ENSG00000186409,ENSG00000117450,ENSG00000186160,ENSG00000162402,ENSG00000117069,ENSG00000122417,ENSG00000069702,ENSG00000079335,ENSG00000163349,...,ENSG00000128228,ENSG00000100154,ENSG00000184949,ENSG00000079974,ENSG00000154723,ENSG00000156273,ENSG00000142168,ENSG00000159079,ENSG00000243927,ENSG00000160180
AAACGCTTCGATACAC-1,0.913277,0.000000,0.000000,2.939676,0.913277,1.700636,0.913277,0.000000,0.913277,0.913277,...,0.000000,1.941602,0.000000,0.000000,0.000000,3.015642,0.913277,0.000000,0.000000,0.000000
AACCAACAGCACCTGC-1,2.104492,1.758195,0.000000,1.758195,0.000000,0.000000,0.788680,0.000000,0.000000,0.788680,...,0.788680,0.788680,0.000000,0.000000,1.224063,1.526375,0.788680,0.000000,0.000000,0.000000
AACGGGACAGCCCAGT-1,0.778024,1.511053,0.000000,3.200440,0.778024,0.000000,0.778024,0.778024,0.000000,0.778024,...,0.000000,0.778024,0.000000,0.778024,0.000000,3.375948,0.000000,0.000000,0.000000,0.000000
AATGGCTCACTTCAGA-1,2.132476,0.000000,1.246623,1.246623,0.602233,0.000000,0.975459,0.000000,1.246623,0.975459,...,0.000000,1.914471,0.000000,0.602233,0.602233,3.036328,1.246623,0.975459,0.975459,0.000000
AATTTCCCACGCGGTT-1,0.862805,0.862805,0.000000,1.318977,0.862805,0.000000,1.868596,0.000000,0.000000,0.000000,...,0.000000,0.862805,0.000000,0.000000,0.862805,3.004552,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGACTCAGCCTATA-9,0.000000,0.000000,0.000000,0.000000,0.971672,1.630396,0.000000,0.000000,0.599484,0.000000,...,0.971672,0.000000,0.000000,0.000000,0.599484,0.599484,0.000000,0.000000,0.971672,0.599484
TTTGACTGTATACAGA-9,0.619643,0.000000,0.000000,0.000000,0.619643,0.000000,0.000000,0.000000,1.666068,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.999355,0.000000,0.619643,0.000000,0.000000
TTTGGAGAGTGATAGT-9,0.592462,0.592462,0.000000,0.592462,1.231191,0.961979,0.592462,1.766546,0.961979,0.592462,...,0.961979,0.000000,0.961979,0.000000,0.592462,0.000000,0.592462,0.000000,0.961979,0.961979
TTTGGAGAGTGCAGCA-9,0.784452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.218589,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.784452,0.000000


In [86]:
gene_list = dataset.columns.tolist();

mg = mygene.MyGeneInfo();

geneSyms = mg.querymany(gene_list, scopes='ensembl.gene', fields='symbol', species='human');

gene_info_dict = {};
for ele in geneSyms:
    # print(ele);
    query = ele['query'];
    if 'symbol' in ele:
        symbol = ele['symbol'];
        gene_info_dict[query] = symbol;
    
dataset.rename(columns=gene_info_dict, inplace=True);

display(dataset);

dataset.to_csv(data_path+file_name+"/processed_data.csv",index=None);

,RPL11,CCDC30,PRDX1,CYP4Z1,USP24,ST6GALNAC5,ODF2L,TGFBR3,CDC14A,HIPK1,...,SDF2L1,TTC28,FAM227A,RABL2B,ATP5PF,BACH1,SOD1,CFAP298,MRPS6,TFF3
AAACGCTTCGATACAC-1,0.913277,0.000000,0.000000,2.939676,0.913277,1.700636,0.913277,0.000000,0.913277,0.913277,...,0.000000,1.941602,0.000000,0.000000,0.000000,3.015642,0.913277,0.000000,0.000000,0.000000
AACCAACAGCACCTGC-1,2.104492,1.758195,0.000000,1.758195,0.000000,0.000000,0.788680,0.000000,0.000000,0.788680,...,0.788680,0.788680,0.000000,0.000000,1.224063,1.526375,0.788680,0.000000,0.000000,0.000000
AACGGGACAGCCCAGT-1,0.778024,1.511053,0.000000,3.200440,0.778024,0.000000,0.778024,0.778024,0.000000,0.778024,...,0.000000,0.778024,0.000000,0.778024,0.000000,3.375948,0.000000,0.000000,0.000000,0.000000
AATGGCTCACTTCAGA-1,2.132476,0.000000,1.246623,1.246623,0.602233,0.000000,0.975459,0.000000,1.246623,0.975459,...,0.000000,1.914471,0.000000,0.602233,0.602233,3.036328,1.246623,0.975459,0.975459,0.000000
AATTTCCCACGCGGTT-1,0.862805,0.862805,0.000000,1.318977,0.862805,0.000000,1.868596,0.000000,0.000000,0.000000,...,0.000000,0.862805,0.000000,0.000000,0.862805,3.004552,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGACTCAGCCTATA-9,0.000000,0.000000,0.000000,0.000000,0.971672,1.630396,0.000000,0.000000,0.599484,0.000000,...,0.971672,0.000000,0.000000,0.000000,0.599484,0.599484,0.000000,0.000000,0.971672,0.599484
TTTGACTGTATACAGA-9,0.619643,0.000000,0.000000,0.000000,0.619643,0.000000,0.000000,0.000000,1.666068,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.999355,0.000000,0.619643,0.000000,0.000000
TTTGGAGAGTGATAGT-9,0.592462,0.592462,0.000000,0.592462,1.231191,0.961979,0.592462,1.766546,0.961979,0.592462,...,0.961979,0.000000,0.961979,0.000000,0.592462,0.000000,0.592462,0.000000,0.961979,0.961979
TTTGGAGAGTGCAGCA-9,0.784452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.218589,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.784452,0.000000
